# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content. 

In [ ]:
!python -m pip install semantic-kernel==0.9.4b1

In [1]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

kernel = sk.Kernel()


from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
service_id = "aoai_chat_completion"
kernel.add_service(
    AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)

Let's define a prompt outlining a dialogue chat bot.

In [3]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [5]:
if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.create_function_from_prompt(function_name="chat",plugin_name="chat",function="prompt",prompt_template_config=prompt_template_config)

In [6]:
chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments

In [7]:
arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot

In [8]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Hello! I'd be happy to help. Could you please tell me what genre or type of book you're interested in? For example, are you looking for fiction, non-fiction, mystery, fantasy, romance, or something else?


Update the history with the output

In [9]:
chat_history.add_assistant_message(str(response))

In [10]:
print(chat_history)

<chat_history><message role="system">You are a helpful chatbot who is good about giving book recommendations.</message><message role="assistant">Hello! I'd be happy to help. Could you please tell me what genre or type of book you're interested in? For example, are you looking for fiction, non-fiction, mystery, fantasy, romance, or something else?</message></chat_history>


Keep Chatting!

In [11]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    chat_history.add_user_message(input_text)

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_assistant_message(str(answer))

In [12]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely, here are a few book recommendations that blend history, philosophy, and Greece:

1. **"The Histories" by Herodotus** - Often called the father of history, Herodotus gives a rich account of the Greco-Persian wars.

2. **"The Iliad and The Odyssey" by Homer** - These epic poems are fundamental to the western canon and are a must-read for anyone interested in Greek history or philosophy. 

3. **"The Peloponnesian War" by Thucydides** - This is a historical account of the Peloponnesian War (431–404 BC), which was fought between the Peloponnesian League (led by Sparta) and the Delian League (led by Athens).

4. **"The Last Days of Socrates" by Plato** - This book contains the dialogues Euthyphro, Apology, Crito, and Phaedo, all of which focus on the days leading up to Socrates' death.

5. **"Alexander of Macedon, 356-323 B.C.: A Historical Biography" by Peter Green** - Thi

In [13]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: I'm glad you found the suggestions interesting! Here's a little more about each book:

1. **"The Histories" by Herodotus** - This book is considered the first work of history in Western literature. It covers the events of the Greco-Persian Wars and also delves into the cultures and geography of the time. 

2. **"The Iliad and The Odyssey" by Homer** - "The Iliad" is an epic poem set during the Trojan War, the siege of Troy by a coalition of Greek states. It focuses on the hero Achilles. "The Odyssey" is about the hero Odysseus's journey home after the Trojan War.

3. **"The Peloponnesian War" by Thucydides** - This is a historical account of the 27-year war between Sparta and Athens, and it offers valuable insights into ancient warfare, politics, and power struggles.

4. **"The Last Days of Socrates" by Plato** - This book is a series of dialogues, which include Socrates's trial and his arguments about the nature of death and th

In [14]:
await chat("if I read The Histories by Herodotus book, what exactly will I learn about Greek history?")

User: if I read The Histories by Herodotus book, what exactly will I learn about Greek history?
ChatBot: "The Histories" by Herodotus is an extensive work and one of the world's earliest comprehensive histories. Here are some of the things you'll learn about:

1. **Background of the Persian Empire**: Herodotus provides a detailed account of the growth of the Persian Empire, its leaders such as Cyrus the Great, Cambyses, and Darius, and the various nations they conquered.

2. **Greco-Persian Wars**: This is the main focus of "The Histories". Herodotus chronicles the conflicts between the Persian Empire and the Greek city-states, including the famous battles of Marathon, Thermopylae, and Salamis.

3. **Geography and Ethnography**: Herodotus describes the lands and cultures of both the Greeks and Persians, along with those of other peoples such as the Egyptians and Scythians. His descriptions are a mix of fact, folklore, and personal observation.

4. **Customs and Practices**: The book pr

In [15]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Of course! Here are a few more books that provide different perspectives and additional information on Greek history and philosophy:

1. **"Persian Fire: The First World Empire and the Battle for the West" by Tom Holland** - This book provides an in-depth view of the Persian Empire and the wars with Greece from the Persian perspective.

2. **"The Oresteia" by Aeschylus** - A trilogy of Greek tragedies that tells the end of the curse on the House of Atreus.

3. **"The Republic" by Plato** - One of the most influential works in philosophy, this book contains discussions on justice, the just man, and the just city-state.

4. **"The Nicomachean Ethics" by Aristotle** - This work by Aristotle examines various areas of life and provides practical advice on how to lead a good life.

5. **"The Rise and Fall of Athens: Nine Greek Lives by Plutarch"** - These biographies provide valuable insight into the period of Greek

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [16]:
print(chat_history)

<chat_history><message role="system">You are a helpful chatbot who is good about giving book recommendations.</message><message role="assistant">Hello! I'd be happy to help. Could you please tell me what genre or type of book you're interested in? For example, are you looking for fiction, non-fiction, mystery, fantasy, romance, or something else?</message><message role="user">I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</message><message role="assistant">Absolutely, here are a few book recommendations that blend history, philosophy, and Greece:

1. **"The Histories" by Herodotus** - Often called the father of history, Herodotus gives a rich account of the Greco-Persian wars.

2. **"The Iliad and The Odyssey" by Homer** - These epic poems are fundamental to the western canon and are a must-read for anyone interested in Greek history or philosophy. 

3. **"The Peloponnesian War" by Thucydides** - This is a historical account of the Peloponn